# Find Candidate Enrichment Biomarkers
Andrew E. Davidson  
aedavids@ucsc.edu   
5/16/24  

Copyright (c) 2020-2023, Regents of the University of California All rights reserved. https://polyformproject.org/licenses/noncommercial/1.0.0

ref: deconvolutionAnalysis/doc/addDegree2Genes.md

<span style="color:red;background-color:yellow">('ESCA', 'STAD') : len(v) = 167</span>  
STAD is 'stomach adenocarcinoma'

In [1]:
import ipynbname

# use display() to print an html version of a data frame
# useful if dataFrame output is not generated by last like of cell
from IPython.display import display

import numpy as np
import os
import pandas as pd

import pprint as pp
import matplotlib.pyplot as plt

import sys

notebookName = ipynbname.name()
notebookPath = ipynbname.path()
notebookDir = os.path.dirname(notebookPath)

#outDir = f'{notebookDir}/{notebookName}.out'
outDir = f'/private/groups/kimlab/aedavids/elife/{notebookName}.out'
os.makedirs(outDir, exist_ok=True)
print(f'outDir:\n{outDir}')

# results of hyperparmeter search
#hyperparameterOut = "/private/groups/kimlab/aedavids/elife/hyperparmeterTunning"

imgOut = f'{outDir}/img'
os.makedirs(imgOut, exist_ok=True)
print(f'\nimgOut :\n{imgOut}')

import logging
loglevel = "INFO"
#loglevel = "WARN"
# logFMT = "%(asctime)s %(levelname)s [thr:%(threadName)s %(name)s %(funcName)s() line:%(lineno)s] [%(message)s]"
logFMT = "%(asctime)s %(levelname)s %(name)s %(funcName)s() line:%(lineno)s] [%(message)s]"
logging.basicConfig(format=logFMT, level=loglevel)    
logger = logging.getLogger(notebookName)

meaningOfLife = 42

/data/tmp/ipykernel_1438096/1756728818.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


outDir:
/private/groups/kimlab/aedavids/elife/findCandidateEnrichmentBiomarkers.out

imgOut :
/private/groups/kimlab/aedavids/elife/findCandidateEnrichmentBiomarkers.out/img


In [2]:
# setting the python path allows us to run python scripts from using
# the CLI. 
ORIG_PYTHONPATH = os.environ['PYTHONPATH']

####### config deconvolutionModules
deconvolutionModules = notebookPath.parent.joinpath("../../../deconvolutionAnalysis/python/")
print("deconvolutionModules: {}\n".format(deconvolutionModules))

PYTHONPATH = ORIG_PYTHONPATH + f':{deconvolutionModules}'
print("PYTHONPATH: {}\n".format(PYTHONPATH))

##### config intraExtraRNA_POCModules
intraExtraRNA_POCModules=notebookPath.parent.joinpath("../../python/src")
print("intraExtraRNA_POCModules: {}\n".format(intraExtraRNA_POCModules))

PYTHONPATH = PYTHONPATH + f':{intraExtraRNA_POCModules}'
print("PYTHONPATH: {}\n".format(PYTHONPATH))

###### set new PYTHONPATH
os.environ["PYTHONPATH"] = PYTHONPATH
PYTHONPATH = os.environ["PYTHONPATH"]
print("PYTHONPATH: {}\n".format(PYTHONPATH))

###### set sys.path
# to be able to import our local python files we need to set the sys.path
# https://stackoverflow.com/a/50155834
sys.path.append( str(deconvolutionModules) )
sys.path.append( str(intraExtraRNA_POCModules) )
print("\nsys.path:\n{}\n".format(sys.path))

deconvolutionModules: /private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning/../../../deconvolutionAnalysis/python

PYTHONPATH: :/private/home/aedavids/extraCellularRNA/src:/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning/../../../deconvolutionAnalysis/python

intraExtraRNA_POCModules: /private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning/../../python/src

PYTHONPATH: :/private/home/aedavids/extraCellularRNA/src:/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning/../../../deconvolutionAnalysis/python:/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning/../../python/src

PYTHONPATH: :/private/home/aedavids/extraCellularRNA/src:/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning/../../../deconvolutionA

In [3]:
# import local 
from analysis.utilities import findIntersectionsWithDegree
from analysis.utilities import loadDictionary

# Find Candidate ESCA Degree 2 Genes
Avoid Genes shared between ESCA and Esophagus

In [4]:
best500LFC_findAllDegree1_wl500Path="/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best500LFC_FindAllDegree1_wl500/training/best500LFC_FindAllDegree1_wl500.sh.out/upsetPlot.out/best500LFC_findAllDegree1_wl500.intersection.dict"
best500LFC_findAllDegree1_wl500_intersectionDict = loadDictionary( best500LFC_findAllDegree1_wl500Path)

In [5]:
degree2Dict = findIntersectionsWithDegree(
                    best500LFC_findAllDegree1_wl500_intersectionDict, 
                    degree=2)

In [6]:
def findBiomarkers(
    intersectionDict : dict[ list, list ], 
    setName: str) -> dict[ list[str], list[str] ]:
    '''
    arguments
        intersectionDict
            key: muliti index of GTEx, or TCGA classes
                example:  ('Liver', 'PRAD', 'UVM')
                
            values: list of biomarkers
                example : ['ABCC11'],
        setName :
            a GTEx, or TCGA classes
                example : 'Liver'

    returns
        a dictionary, all the keys will contain setName
    '''

    retDict = dict()
    for key,values in intersectionDict.items():
        if setName in key:
            retDict[key] = values

    return retDict

In [7]:
ESCAIntersectionDict = findBiomarkers(degree2Dict, "ESCA")

def viewDict( intersectionDict ) :
    for k,v in intersectionDict.items():    
        if len(v) < 5:
            print(f'{str(k)} : {v}')
        else :
            print(f'{k} : len(v) = {len(v)}')

viewDict( ESCAIntersectionDict )

('ESCA', 'Esophagus_Mucosa') : ['TMPRSS11BNL', 'KRT24', '(TGGCCC)n']
('ESCA', 'Lung') : ['AC108058.1']
('ESCA', 'Minor_Salivary_Gland') : ['ANKRD36C']
('ESCA', 'OV') : ['GAL3ST2', 'AC142086.6', 'AL031708.1']
('ESCA', 'PAAD') : ['FGF19']
('ESCA', 'Prostate') : ['(GTGTCGT)n']
('ESCA', 'SARC') : ['AC002550.1', 'Arthur1A']
('ESCA', 'STAD') : len(v) = 167
('ESCA', 'Testis') : ['AC007598.2']


## Compare Differential Expression Values

In [8]:
def findCandidateBiomarkers(
    intersectionDict : dict[ list, list ],
    ignore : list[str],
    ):
    '''
    TODO
    '''
    retList = []
    ignoreSet = set(ignore)
    #t is tuple of set names
    for t,v in intersectionDict.items():
        setName = set(t)
        if len( setName.intersection(ignoreSet) ) == 0:
            print(f'adding biomarkers from {setName}')
            retList = retList + v

    return retList

xxx = findCandidateBiomarkers( ESCAIntersectionDict, ignore=['Esophagus_Mucosa', 'STAD'])
xxx

adding biomarkers from {'Lung', 'ESCA'}
adding biomarkers from {'Minor_Salivary_Gland', 'ESCA'}
adding biomarkers from {'OV', 'ESCA'}
adding biomarkers from {'PAAD', 'ESCA'}
adding biomarkers from {'Prostate', 'ESCA'}
adding biomarkers from {'SARC', 'ESCA'}
adding biomarkers from {'Testis', 'ESCA'}


['AC108058.1',
 'ANKRD36C',
 'GAL3ST2',
 'AC142086.6',
 'AL031708.1',
 'FGF19',
 '(GTGTCGT)n',
 'AC002550.1',
 'Arthur1A',
 'AC007598.2']